In [1]:
# # Note-Point
# Sunday -> Data Visualization and EDA Analysis of data +  Understanding of the data 
# Monday -> Fitting the Model 
# Tuesday -> Evaluation of the Model 
# Wednessday -> Report submit it.

# Libraries

In [2]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from statsmodels.tsa.stattools import adfuller

# Import libraries from darts package
from darts import TimeSeries
from darts import TimeSeries as tm 
from darts.models.forecasting.auto_arima import AutoARIMA
from darts.models.forecasting.varima import VARIMA
from darts.dataprocessing.transformers.scaler import Scaler

# Import evaluation metrics from darts package
from darts.metrics.metrics import mape, mase, mse, rmse, smape, mae

# Ignore warnings in the code
# warnings.filterwarnings("ignore")

/home/ifti/anaconda3/envs/darts/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/ifti/anaconda3/envs/darts/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZNK3c1010TensorImpl36is_contiguous_nondefault_policy_implENS_12MemoryFormatE
  warn(f"Failed to load image Python extension: {e}")


# Helping Rotuines 

In [14]:
# import adfuller from statsmodels.tsa.stattools
# import mae, smape, mse, rmse, mape, mase from darts.metrics.metrics

# Function to visualize time series data
def Visualize(date_index, value, title:str):
    # Create a Plotly figure
    fig = go.Figure()
    
    # Add a trace to the figure
    fig.add_trace(go.Scatter(x=date_index, y=value, name=title))
    
    # Update trace information and layout
    fig.update_traces(hoverinfo='text+name', mode='lines+markers')
    fig.update_layout(legend=dict(y=0.1, traceorder='reversed', font_size=16))
    fig.update_layout(title=str(title))
    
    # Enable range selector for the x-axis
    fig.update_xaxes(rangeslider_visible=True,
                     rangeselector=dict(buttons=list([dict(step="all")])))

    # Show the plot
    fig.show()

# Function to perform AD Fuller test on a time series data
def adfuller_test(shot_count):
    # Perform the AD Fuller test
    result = adfuller(shot_count)
    
    # Print the test results
    labels = ['ADF Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used']
    for value, label in zip(result, labels):
        print(label + ' : ' + str(value))
        
    # Interpret the test results
    if result[1] <= 0.05:
        print("strong evidence against the null hypothesis(Ho), reject the null hypothesis. Data has no unit root and is stationary")
    else:
        print("weak evidence against null hypothesis, time series has a unit root, indicating it is non-stationary ")

# Function to calculate evaluation metrics for time series forecasts
def Evaluation_matrics(actual, predicted, insample):
    # Calculate the evaluation metrics
    return {
        'mae': mae(actual, predicted),
        'sampe': smape(actual, predicted),
        "mse": mse(actual, predicted),
        "rmse": rmse(actual, predicted),
        "mape": mape(actual, predicted),
        "mase": mase(actual, predicted, insample)

    }


def multivariate_series(df ,site_name = None , aqs_param=None):
    site_leve_1 = df[df['Site Name']==site_name]
    aqs_level_2 = site_leve_1[site_leve_1['AQS_PARAMETER_DESC']==aqs_param]
    grouped = aqs_level_2[['Date',  'DAILY_AQI_VALUE']]
    grouped.reset_index(inplace=True)
    return grouped


# EDA Analysis

In [4]:
air_pol = pd.read_csv('full.csv',index_col='Date', parse_dates=True)
air_pol =  air_pol[['UNITS', 'DAILY_AQI_VALUE', 'Site Name', 'AQS_PARAMETER_DESC','COUNTY', 'SITE_LATITUDE', 'SITE_LONGITUDE']]
air_pol['AQS_PARAMETER_DESC']

Date
2017-01-01             Carbon monoxide
2017-01-02             Carbon monoxide
2017-01-03             Carbon monoxide
2017-01-04             Carbon monoxide
2017-01-05             Carbon monoxide
                        ...           
2021-12-27    PM2.5 - Local Conditions
2021-12-28    PM2.5 - Local Conditions
2021-12-29    PM2.5 - Local Conditions
2021-12-30    PM2.5 - Local Conditions
2021-12-31    PM2.5 - Local Conditions
Name: AQS_PARAMETER_DESC, Length: 109994, dtype: object

In [5]:
air_pol = air_pol[ (air_pol['Site Name']=='Azusa' ) & ((air_pol['AQS_PARAMETER_DESC']=='Ozone' ) |  (air_pol['AQS_PARAMETER_DESC']=='PM2.5 - Local Conditions' ))  ]
air_pol

,UNITS,DAILY_AQI_VALUE,Site Name,AQS_PARAMETER_DESC,COUNTY,SITE_LATITUDE,SITE_LONGITUDE
Date,,,,,,,
2017-01-01,ug/m3 LC,35.0,Azusa,PM2.5 - Local Conditions,Los Angeles,34.1365,-117.92391
2017-01-04,ug/m3 LC,35.0,Azusa,PM2.5 - Local Conditions,Los Angeles,34.1365,-117.92391
2017-01-07,ug/m3 LC,15.0,Azusa,PM2.5 - Local Conditions,Los Angeles,34.1365,-117.92391
2017-01-10,ug/m3 LC,21.0,Azusa,PM2.5 - Local Conditions,Los Angeles,34.1365,-117.92391
2017-01-13,ug/m3 LC,19.0,Azusa,PM2.5 - Local Conditions,Los Angeles,34.1365,-117.92391
...,...,...,...,...,...,...,...
2021-12-27,ppm,24.0,Azusa,Ozone,Los Angeles,34.1365,-117.92391
2021-12-28,ppm,28.0,Azusa,Ozone,Los Angeles,34.1365,-117.92391
2021-12-29,ppm,28.0,Azusa,Ozone,Los Angeles,34.1365,-117.92391


## Visualization of the Data

In [6]:
grouped = air_pol.groupby(['Site Name','AQS_PARAMETER_DESC'])
for name ,group in grouped:
    Visualize(group.index,group['DAILY_AQI_VALUE'],title=str(name))
    break


## Univariate Series

In [11]:
# def univariate_series(df ,site_name = None , aqs_param_1=None,aqs_param_2=None):
#     df = df[ (df['Site Name']=='Azusa' ) & ((df['AQS_PARAMETER_DESC']==aqs_param_1 ) |  (df['AQS_PARAMETER_DESC']==aqs_param_2 ))  ]
#     df.reset_index(inplace=True)
#     grouped = df[['Date','DAILY_AQI_VALUE']]
#     return grouped
    


                    
# df =  pd.read_csv('full.csv')
# df = univariate_series(df,
#                 site_name = 'Los Angeles-North Main Street',
#                 aqs_param_1 = 'Ozone',
#                 aqs_param_2 = 'PM2.5 - Local Conditions',)

# # df_series = TimeSeries.from_dataframe(df,time_col='Date',freq='1D')
# # df_series_2 =  TimeSeries.from_dataframe(df,time_col='Date',freq='1D',fill_missing_dates=True,fillna_value=True)

# ARIMA Model

In [41]:
from darts.models.forecasting.auto_arima import AutoARIMA

df =  pd.read_csv('full.csv')
df = multivariate_series(df,
                site_name = 'Los Angeles-North Main Street',
                aqs_param = 'Ozone')
df = df.drop(columns=['index'])
df_series_2 =  TimeSeries.from_dataframe(df,time_col='Date',freq='1D',fill_missing_dates=True,fillna_value=True)

train_series ,testing_sc = df_series_2.split_after(0.8)

arima =  AutoARIMA(add_encoders={
    'cyclic': {'future': ['month']},
    'datetime_attribute': {'future': ['hour', 'dayofweek']},
    'position': {'future': ['relative']},
    'custom': {'future': [lambda idx: (idx.year - 1950) / 50]},
    'transformer': Scaler()
})
arima.fit(train_series)

/home/ifti/anaconda3/envs/darts/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning:

divide by zero encountered in reciprocal

/home/ifti/anaconda3/envs/darts/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:1906: RuntimeWarning:

divide by zero encountered in reciprocal

/home/ifti/anaconda3/envs/darts/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning:

divide by zero encountered in reciprocal

/home/ifti/anaconda3/envs/darts/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:1906: RuntimeWarning:

divide by zero encountered in reciprocal

/home/ifti/anaconda3/envs/darts/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning:

divide by zero encountered in reciprocal

/home/ifti/anaconda3/envs/darts/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:1906: RuntimeWarning:

divide by zero encountered in reciprocal

/home/ifti/anaco

## Evaluation of AQS parameter of Ozone

In [42]:
# Define a list of horizons to be used for forecasting
Horizan = [30, 90, 180, 365]

# Create an empty list to store the evaluation results for each horizon
eval_list = []

# Loop through each horizon in the list
for horizan in Horizan:

    # Predict the AQI value for the given horizon using XGBoost
    predicted = arima.predict(horizan)

    # Calculate the evaluation metrics for the predicted values and actual values
    ret_eval_dic = Evaluation_matrics(testing_sc[0:horizan],predicted,train_series)
    # Store the evaluation results for the current horizon in a dictionary
    eval_dic = {"Horizon": horizan,
                "MAE": ret_eval_dic['mae'],
                "SMAPE": ret_eval_dic['sampe'],
                "MSE": ret_eval_dic['mse'],
                "RMSE": ret_eval_dic['rmse'],
                "MAPE": ret_eval_dic['mape']
                }

    # Add the evaluation results for the current horizon to the list of results
    eval_list.append(eval_dic)

# Create a Pandas dataframe from the evaluation results
ev_df = pd.DataFrame(eval_list)

# Display the evaluation results dataframe
ev_df


,Horizon,MAE,SMAPE,MSE,RMSE,MAPE
0,30,27.565492,164.017980,847.900392,29.118729,116.512022
1,90,20.382708,94.200541,517.720767,22.753478,67.206070
2,180,15.970406,59.267545,370.514219,19.248746,70.546455
3,365,16.289525,50.072103,415.904153,20.393728,166.178043


#### Evaluation of Horizan is 1 Month

In [43]:
# Predicting AQI values for 30 days horizon
Horizan = 30

# Fitting XGBoost model and predicting AQI values for the given horizon
predicted = arima.predict(Horizan)

# Evaluating the prediction performance of the XGBoost model 
print(Evaluation_matrics(testing_sc[0:Horizan],predicted,train_series))

# Creating visualization of predicted and actual AQI values
fig = go.Figure()

# Adding a trace for predicted AQI values
fig.add_trace(  go.Scatter(
                            x=predicted.pd_series().index, # x axis values set to index of predicted series
                            y=predicted.pd_series().values, # y axis values set to values of predicted series
                            name='predicted'))

# Adding a trace for actual AQI values
fig.add_trace(  go.Scatter(
                            x=testing_sc[0:Horizan].pd_series().index, # x axis values set to index of actual series
                            y=testing_sc[0:Horizan].pd_series().values, # y axis values set to values of actual series
                            name='actual'))

# Updating the layout of the visualization
fig.update_traces(hoverinfo='text+name', mode='lines+markers') # Adding hover info to show text and name of each trace
fig.update_layout(legend=dict(y=0.1, traceorder='reversed', font_size=16)) # Updating the legend of the visualization
fig.update_layout(title="Horizon (Days) : "+ str(Horizan)) # Updating the title of the visualization

# Updating the x-axis of the visualization to have rangeslider and rangeselector
fig.update_xaxes(
                rangeslider_visible=True,
                rangeselector=dict(
                buttons=list([dict(step="all")])
                ))

# Showing the final visualization
fig.show()


{'mae': 27.56549206903094, 'sampe': 164.01797975426064, 'mse': 847.9003922899734, 'rmse': 29.11872923549332, 'mape': 116.51202154884996, 'mase': 2.472697125877683}


#### Evaluation of Horizan is 3 Month


In [40]:
# Predicting AQI values for 30 days horizon
Horizan = 30*3

# Fitting XGBoost model and predicting AQI values for the given horizon
predicted = arima.predict(Horizan)

# Evaluating the prediction performance of the XGBoost model 
print(Evaluation_matrics(testing_sc[0:Horizan],predicted,train_series))

# Creating visualization of predicted and actual AQI values
fig = go.Figure()

# Adding a trace for predicted AQI values
fig.add_trace(  go.Scatter(
                            x=predicted.pd_series().index, # x axis values set to index of predicted series
                            y=predicted.pd_series().values, # y axis values set to values of predicted series
                            name='predicted'))

# Adding a trace for actual AQI values
fig.add_trace(  go.Scatter(
                            x=testing_sc[0:Horizan].pd_series().index, # x axis values set to index of actual series
                            y=testing_sc[0:Horizan].pd_series().values, # y axis values set to values of actual series
                            name='actual'))

# Updating the layout of the visualization
fig.update_traces(hoverinfo='text+name', mode='lines+markers') # Adding hover info to show text and name of each trace
fig.update_layout(legend=dict(y=0.1, traceorder='reversed', font_size=16)) # Updating the legend of the visualization
fig.update_layout(title="Horizon (Days) : "+ str(Horizan)) # Updating the title of the visualization

# Updating the x-axis of the visualization to have rangeslider and rangeselector
fig.update_xaxes(
                rangeslider_visible=True,
                rangeselector=dict(
                buttons=list([dict(step="all")])
                ))

# Showing the final visualization
fig.show()


{'mae': 21.498791073331855, 'sampe': 99.09801966421698, 'mse': 564.8067682767037, 'rmse': 23.765663640569848, 'mape': 70.08998584099156, 'mase': 1.928498093331562}


#### Evaluation of Horizan is 6 Month


In [36]:
# Predicting AQI values for 30 days horizon
Horizan = 30*6

# Fitting XGBoost model and predicting AQI values for the given horizon
predicted = arima.predict(Horizan)

# Evaluating the prediction performance of the XGBoost model 
print(Evaluation_matrics(testing_sc[0:Horizan],predicted,train_series))

# Creating visualization of predicted and actual AQI values
fig = go.Figure()

# Adding a trace for predicted AQI values
fig.add_trace(  go.Scatter(
                            x=predicted.pd_series().index, # x axis values set to index of predicted series
                            y=predicted.pd_series().values, # y axis values set to values of predicted series
                            name='predicted'))

# Adding a trace for actual AQI values
fig.add_trace(  go.Scatter(
                            x=testing_sc[0:Horizan].pd_series().index, # x axis values set to index of actual series
                            y=testing_sc[0:Horizan].pd_series().values, # y axis values set to values of actual series
                            name='actual'))

# Updating the layout of the visualization
fig.update_traces(hoverinfo='text+name', mode='lines+markers') # Adding hover info to show text and name of each trace
fig.update_layout(legend=dict(y=0.1, traceorder='reversed', font_size=16)) # Updating the legend of the visualization
fig.update_layout(title="Horizon (Days) : "+ str(Horizan)) # Updating the title of the visualization

# Updating the x-axis of the visualization to have rangeslider and rangeselector
fig.update_xaxes(
                rangeslider_visible=True,
                rangeselector=dict(
                buttons=list([dict(step="all")])
                ))

# Showing the final visualization
fig.show()


{'mae': 11.762015753508184, 'sampe': 32.62895210503088, 'mse': 221.2321079098288, 'rmse': 14.873873332452069, 'mape': 57.57683276366481, 'mase': 1.0550837429418742}


#### Evaluation of Horizan is 1 Year


In [28]:
# Predicting AQI values for 30 days horizon
Horizan = 30*12

# Fitting XGBoost model and predicting AQI values for the given horizon
predicted = arima.predict(Horizan)

# Evaluating the prediction performance of the XGBoost model 
print(Evaluation_matrics(testing_sc[0:Horizan],predicted,train_series))

# Creating visualization of predicted and actual AQI values
fig = go.Figure()

# Adding a trace for predicted AQI values
fig.add_trace(  go.Scatter(
                            x=predicted.pd_series().index, # x axis values set to index of predicted series
                            y=predicted.pd_series().values, # y axis values set to values of predicted series
                            name='predicted'))

# Adding a trace for actual AQI values
fig.add_trace(  go.Scatter(
                            x=testing_sc[0:Horizan].pd_series().index, # x axis values set to index of actual series
                            y=testing_sc[0:Horizan].pd_series().values, # y axis values set to values of actual series
                            name='actual'))

# Updating the layout of the visualization
fig.update_traces(hoverinfo='text+name', mode='lines+markers') # Adding hover info to show text and name of each trace
fig.update_layout(legend=dict(y=0.1, traceorder='reversed', font_size=16)) # Updating the legend of the visualization
fig.update_layout(title="Horizon (Days) : "+ str(Horizan)) # Updating the title of the visualization

# Updating the x-axis of the visualization to have rangeslider and rangeselector
fig.update_xaxes(
                rangeslider_visible=True,
                rangeselector=dict(
                buttons=list([dict(step="all")])
                ))

# Showing the final visualization
fig.show()


{'mae': 10.682769015544713, 'sampe': 29.08272654429993, 'mse': 243.5486330018493, 'rmse': 15.606044758421312, 'mape': 136.92169672778437, 'mase': 0.9582724725175276}


# ARIMA Model forcasting AQI value of PM2.5 - Local Conditions

In [44]:
# # Importing the full.csv file into pandas dataframe
df =  pd.read_csv('full.csv')

# Filter dataframe with specific site_name and aqs_param
df = multivariate_series(df,
                site_name = 'Los Angeles-North Main Street',
                aqs_param = 'PM2.5 - Local Conditions')

# Drop columns 'index' and duplicate dates
df = df.drop(columns=['index'])
df = df.drop_duplicates(subset='Date')

# Convert pandas dataframe to TimeSeries object
df_series_2 =  TimeSeries.from_dataframe(df,time_col='Date',freq='1D',fill_missing_dates=True,fillna_value=True)

# Split TimeSeries data into train and test sets
train_series ,testing_sc = df_series_2.split_after(0.8)

arima_pm =  AutoARIMA(add_encoders={
    'cyclic': {'future': ['month']},
    'datetime_attribute': {'future': ['hour', 'dayofweek']},
    'position': {'future': ['relative']},
    'custom': {'future': [lambda idx: (idx.year - 1950) / 50]},
    'transformer': Scaler()
})
arima_pm.fit(train_series)

## Evaluation of ARIMA Model of PM2

In [45]:
# Define a list of horizons to be used for forecasting
Horizan = [30, 90, 180, 365]

# Create an empty list to store the evaluation results for each horizon
eval_list = []

# Loop through each horizon in the list
for horizan in Horizan:

    # Predict the AQI value for the given horizon using XGBoost
    predicted = arima_pm.predict(horizan)

    # Calculate the evaluation metrics for the predicted values and actual values
    ret_eval_dic = Evaluation_matrics(testing_sc[0:horizan],predicted,train_series)
    # Store the evaluation results for the current horizon in a dictionary
    eval_dic = {"Horizon": horizan,
                "MAE": ret_eval_dic['mae'],
                "SMAPE": ret_eval_dic['sampe'],
                "MSE": ret_eval_dic['mse'],
                "RMSE": ret_eval_dic['rmse'],
                "MAPE": ret_eval_dic['mape']
                }

    # Add the evaluation results for the current horizon to the list of results
    eval_list.append(eval_dic)

# Create a Pandas dataframe from the evaluation results
ev_df = pd.DataFrame(eval_list)

# Display the evaluation results dataframe
ev_df


,Horizon,MAE,SMAPE,MSE,RMSE,MAPE
0,30,14.964516,35.674941,298.945731,17.290047,47.482631
1,90,12.601849,32.242680,244.049270,15.622076,43.197055
2,180,12.249386,30.318944,222.766367,14.925360,37.283325
3,365,14.858209,30.932409,475.169713,21.798388,51.226892


#### Evaluation of Horizan is 1 Month


In [46]:
# Predicting AQI values for 30 days horizon
Horizan = 30

# Fitting XGBoost model and predicting AQI values for the given horizon
predicted = arima_pm.predict(Horizan)

# Evaluating the prediction performance of the XGBoost model 
print(Evaluation_matrics(testing_sc[0:Horizan],predicted,train_series))

# Creating visualization of predicted and actual AQI values
fig = go.Figure()

# Adding a trace for predicted AQI values
fig.add_trace(  go.Scatter(
                            x=predicted.pd_series().index, # x axis values set to index of predicted series
                            y=predicted.pd_series().values, # y axis values set to values of predicted series
                            name='predicted'))

# Adding a trace for actual AQI values
fig.add_trace(  go.Scatter(
                            x=testing_sc[0:Horizan].pd_series().index, # x axis values set to index of actual series
                            y=testing_sc[0:Horizan].pd_series().values, # y axis values set to values of actual series
                            name='actual'))

# Updating the layout of the visualization
fig.update_traces(hoverinfo='text+name', mode='lines+markers') # Adding hover info to show text and name of each trace
fig.update_layout(legend=dict(y=0.1, traceorder='reversed', font_size=16)) # Updating the legend of the visualization
fig.update_layout(title="Horizon (Days) : "+ str(Horizan)) # Updating the title of the visualization

# Updating the x-axis of the visualization to have rangeslider and rangeselector
fig.update_xaxes(
                rangeslider_visible=True,
                rangeselector=dict(
                buttons=list([dict(step="all")])
                ))

# Showing the final visualization
fig.show()


{'mae': 14.964515803328585, 'sampe': 35.67494054199712, 'mse': 298.94573074794846, 'rmse': 17.29004715863865, 'mape': 47.482631411756536, 'mase': 1.261078965244429}


#### Evaluation of Horizan is 3 Month


In [47]:
# Predicting AQI values for 30 days horizon
Horizan = 30*3

# Fitting XGBoost model and predicting AQI values for the given horizon
predicted = arima.predict(Horizan)

# Evaluating the prediction performance of the XGBoost model 
print(Evaluation_matrics(testing_sc[0:Horizan],predicted,train_series))

# Creating visualization of predicted and actual AQI values
fig = go.Figure()

# Adding a trace for predicted AQI values
fig.add_trace(  go.Scatter(
                            x=predicted.pd_series().index, # x axis values set to index of predicted series
                            y=predicted.pd_series().values, # y axis values set to values of predicted series
                            name='predicted'))

# Adding a trace for actual AQI values
fig.add_trace(  go.Scatter(
                            x=testing_sc[0:Horizan].pd_series().index, # x axis values set to index of actual series
                            y=testing_sc[0:Horizan].pd_series().values, # y axis values set to values of actual series
                            name='actual'))

# Updating the layout of the visualization
fig.update_traces(hoverinfo='text+name', mode='lines+markers') # Adding hover info to show text and name of each trace
fig.update_layout(legend=dict(y=0.1, traceorder='reversed', font_size=16)) # Updating the legend of the visualization
fig.update_layout(title="Horizon (Days) : "+ str(Horizan)) # Updating the title of the visualization

# Updating the x-axis of the visualization to have rangeslider and rangeselector
fig.update_xaxes(
                rangeslider_visible=True,
                rangeselector=dict(
                buttons=list([dict(step="all")])
                ))

# Showing the final visualization
fig.show()


{'mae': 27.059248896659494, 'sampe': 96.59684924762145, 'mse': 1188.8903145439253, 'rmse': 34.480288782780306, 'mape': 65.41529739199466, 'mase': 2.280317655937827}


#### Evaluation of Horizan is 6 Month


In [48]:
# Predicting AQI values for 30 days horizon
Horizan = 30*6

# Fitting XGBoost model and predicting AQI values for the given horizon
predicted = arima.predict(Horizan)

# Evaluating the prediction performance of the XGBoost model 
print(Evaluation_matrics(testing_sc[0:Horizan],predicted,train_series))

# Creating visualization of predicted and actual AQI values
fig = go.Figure()

# Adding a trace for predicted AQI values
fig.add_trace(  go.Scatter(
                            x=predicted.pd_series().index, # x axis values set to index of predicted series
                            y=predicted.pd_series().values, # y axis values set to values of predicted series
                            name='predicted'))

# Adding a trace for actual AQI values
fig.add_trace(  go.Scatter(
                            x=testing_sc[0:Horizan].pd_series().index, # x axis values set to index of actual series
                            y=testing_sc[0:Horizan].pd_series().values, # y axis values set to values of actual series
                            name='actual'))

# Updating the layout of the visualization
fig.update_traces(hoverinfo='text+name', mode='lines+markers') # Adding hover info to show text and name of each trace
fig.update_layout(legend=dict(y=0.1, traceorder='reversed', font_size=16)) # Updating the legend of the visualization
fig.update_layout(title="Horizon (Days) : "+ str(Horizan)) # Updating the title of the visualization

# Updating the x-axis of the visualization to have rangeslider and rangeselector
fig.update_xaxes(
                rangeslider_visible=True,
                rangeselector=dict(
                buttons=list([dict(step="all")])
                ))

# Showing the final visualization
fig.show()


{'mae': 20.65894624587096, 'sampe': 64.38973025456544, 'mse': 739.5041143716342, 'rmse': 27.193824930885214, 'mape': 53.32819457102733, 'mase': 1.7409559318309726}


#### Evaluation of Horizan is 1 Year


In [49]:
# Predicting AQI values for 30 days horizon
Horizan = 30*12

# Fitting XGBoost model and predicting AQI values for the given horizon
predicted = arima.predict(Horizan)

# Evaluating the prediction performance of the XGBoost model 
print(Evaluation_matrics(testing_sc[0:Horizan],predicted,train_series))

# Creating visualization of predicted and actual AQI values
fig = go.Figure()

# Adding a trace for predicted AQI values
fig.add_trace(  go.Scatter(
                            x=predicted.pd_series().index, # x axis values set to index of predicted series
                            y=predicted.pd_series().values, # y axis values set to values of predicted series
                            name='predicted'))

# Adding a trace for actual AQI values
fig.add_trace(  go.Scatter(
                            x=testing_sc[0:Horizan].pd_series().index, # x axis values set to index of actual series
                            y=testing_sc[0:Horizan].pd_series().values, # y axis values set to values of actual series
                            name='actual'))

# Updating the layout of the visualization
fig.update_traces(hoverinfo='text+name', mode='lines+markers') # Adding hover info to show text and name of each trace
fig.update_layout(legend=dict(y=0.1, traceorder='reversed', font_size=16)) # Updating the legend of the visualization
fig.update_layout(title="Horizon (Days) : "+ str(Horizan)) # Updating the title of the visualization

# Updating the x-axis of the visualization to have rangeslider and rangeselector
fig.update_xaxes(
                rangeslider_visible=True,
                rangeselector=dict(
                buttons=list([dict(step="all")])
                ))

# Showing the final visualization
fig.show()


{'mae': 19.695073715313537, 'sampe': 49.063471518837886, 'mse': 749.9712659197966, 'rmse': 27.385603260103593, 'mape': 58.80821401300231, 'mase': 1.659729155807086}


# XGBOOST


In [12]:

# df =  pd.read_csv('full.csv')
# df = univariate_series(df,
#                 site_name = 'Azusa',
#                 aqs_param = 'Ozone')

# df_series = TimeSeries.from_dataframe(df,time_col='Date',freq='1D')
# df_series_2 =  TimeSeries.from_dataframe(df,time_col='Date',freq='1D',fill_missing_dates=True,fillna_value=True)


# Forcasting AQI value of 'Los Angeles-North Main Street

In [10]:
from darts.models.forecasting.xgboost import XGBModel

In [13]:
df =  pd.read_csv('full.csv')
df = multivariate_series(df,
                site_name = 'Los Angeles-North Main Street',
                aqs_param = 'Ozone')
df = df.drop(columns=['index'])
df_series_2 =  TimeSeries.from_dataframe(df,time_col='Date',freq='1D',fill_missing_dates=True,fillna_value=True)


train_series ,testing_sc = df_series_2.split_after(0.8)

print("Shape of Training Data(In Days) : ",train_series.pd_dataframe().shape)
print("Shape of Testing Data(In Days) : ",testing_sc.pd_dataframe().shape)



# When Fitting the XGBMODEL most important parameters
    # Number of lags do you want to check the before the inference time or start of prediction
    # Output_chunk_lenght == Horizan or Number of prediction in day you want to predict
xgb = XGBModel(lags=90, 
               lags_past_covariates=None, 
               lags_future_covariates=None, 
               output_chunk_length=30, 
               add_encoders=None, 
               )


xgb.fit(train_series)



Shape of Training Data(In Days) :  (1461, 1)
Shape of Testing Data(In Days) :  (365, 1)


In [12]:
# Define a list of horizons to be used for forecasting
Horizan = [30, 90, 180, 365]

# Create an empty list to store the evaluation results for each horizon
eval_list = []

# Loop through each horizon in the list
for horizan in Horizan:

    # Predict the AQI value for the given horizon using XGBoost
    predicted = xgb.predict(horizan)

    # Calculate the evaluation metrics for the predicted values and actual values
    ret_eval_dic = Evaluation_matrics(testing_sc[0:horizan],predicted,train_series)
    # Store the evaluation results for the current horizon in a dictionary
    eval_dic = {"Horizon": horizan,
                "MAE": ret_eval_dic['mae'],
                "SMAPE": ret_eval_dic['sampe'],
                "MSE": ret_eval_dic['mse'],
                "RMSE": ret_eval_dic['rmse'],
                "MAPE": ret_eval_dic['mape']
                }

    # Add the evaluation results for the current horizon to the list of results
    eval_list.append(eval_dic)

# Create a Pandas dataframe from the evaluation results
ev_df = pd.DataFrame(eval_list)

# Display the evaluation results dataframe
ev_df


,Horizon,MAE,SMAPE,MSE,RMSE,MAPE
0,30,9.792015,42.331676,127.312443,11.283282,112.448958
1,90,12.134547,40.484228,201.166575,14.183320,58.536337
2,180,12.921488,36.932268,256.267653,16.008362,57.939425
3,365,13.401286,38.320592,322.997392,17.972128,114.477528


## Evaluation of Horizan is 1 montn

In [13]:
# Predicting AQI values for 30 days horizon
Horizan = 30

# Fitting XGBoost model and predicting AQI values for the given horizon
predicted = xgb.predict(Horizan)

# Evaluating the prediction performance of the XGBoost model 
print(Evaluation_matrics(testing_sc[0:Horizan],predicted,train_series))

# Creating visualization of predicted and actual AQI values
fig = go.Figure()

# Adding a trace for predicted AQI values
fig.add_trace(  go.Scatter(
                            x=predicted.pd_series().index, # x axis values set to index of predicted series
                            y=predicted.pd_series().values, # y axis values set to values of predicted series
                            name='predicted'))

# Adding a trace for actual AQI values
fig.add_trace(  go.Scatter(
                            x=testing_sc[0:Horizan].pd_series().index, # x axis values set to index of actual series
                            y=testing_sc[0:Horizan].pd_series().values, # y axis values set to values of actual series
                            name='actual'))

# Updating the layout of the visualization
fig.update_traces(hoverinfo='text+name', mode='lines+markers') # Adding hover info to show text and name of each trace
fig.update_layout(legend=dict(y=0.1, traceorder='reversed', font_size=16)) # Updating the legend of the visualization
fig.update_layout(title="Horizon (Days) : "+ str(Horizan)) # Updating the title of the visualization

# Updating the x-axis of the visualization to have rangeslider and rangeselector
fig.update_xaxes(
                rangeslider_visible=True,
                rangeselector=dict(
                buttons=list([dict(step="all")])
                ))

# Showing the final visualization
fig.show()


{'mae': 9.792014980316162, 'sampe': 42.331676182743564, 'mse': 127.31244268581767, 'rmse': 11.283281556613646, 'mape': 112.44895801575873, 'mase': 0.8783694931962152}


## Evaluation of Horizan is 3 montn

In [14]:
Horizan = 30*3
# auto_arima.fit(train_sc)
predicted = xgb.predict(Horizan)
print(Evaluation_matrics(testing_sc[0:Horizan],predicted,train_series))

# Create traces
fig = go.Figure()

fig.add_trace(  go.Scatter(
                            x=predicted.pd_series().index,
                            y=predicted.pd_series().values,
                            name='predicted'))

fig.add_trace(  go.Scatter(
                            x=testing_sc[0:Horizan].pd_series().index,
                            y=testing_sc[0:Horizan].pd_series().values,
                            name='actual'))




fig.update_traces(hoverinfo='text+name', mode='lines+markers')
fig.update_layout(legend=dict(y=0.1, traceorder='reversed', font_size=16))
fig.update_layout(title="Horizan(Days) : "+ str(Horizan))


fig.update_xaxes(
                rangeslider_visible=True,
                rangeselector=dict(
                buttons=list([dict(step="all")])
                ))


fig.show()

{'mae': 12.13454729715983, 'sampe': 40.48422775906863, 'mse': 201.1665748808752, 'rmse': 14.183320305234426, 'mape': 58.53633736323409, 'mase': 1.0885008020308031}


## Evaluation of Horizan is 6 montn

In [15]:
# Define the horizon as 30 * 6 = 180 days
Horizan = 30*6

# Run prediction on the input horizon
predicted = xgb.predict(Horizan)

# Print the evaluation metrics for the predicted values and actual values in the testing data
print(Evaluation_matrics(testing_sc[0:Horizan],predicted,train_series))

# Create traces
fig = go.Figure()

# Add trace for the predicted values
fig.add_trace(  go.Scatter(
                            x=predicted.pd_series().index,
                            y=predicted.pd_series().values,
                            name='predicted'))

# Add trace for the actual values
fig.add_trace(  go.Scatter(
                            x=testing_sc[0:Horizan].pd_series().index,
                            y=testing_sc[0:Horizan].pd_series().values,
                            name='actual'))

# Update the hover information for both traces
fig.update_traces(hoverinfo='text+name', mode='lines+markers')

# Update the layout for the legend with the specified font size and trace order
fig.update_layout(legend=dict(y=0.1, traceorder='reversed', font_size=16))

# Update the layout for the title with the horizon value
fig.update_layout(title="Horizan(Days) : "+ str(Horizan))

# Update the x axis with the specified range selector
fig.update_xaxes(
                rangeslider_visible=True,
                rangeselector=dict(
                buttons=list([dict(step="all")])
                ))

# Show the figure
fig.show()


{'mae': 12.92148846520318, 'sampe': 36.93226846396481, 'mse': 256.2676530830987, 'rmse': 16.00836197376542, 'mape': 57.93942513844813, 'mase': 1.1590914941752668}


## Evaluation of Horizan is 1 Year

In [16]:
# Set horizon to 30 * 12
Horizan = 30 * 12

# Use XGBoost to make a prediction for the time series data with the specified horizon
predicted = xgb.predict(Horizan)

# Evaluate the prediction using the Evaluation_matrics function, which likely calculates some performance metrics
print(Evaluation_matrics(testing_sc[0:Horizan],predicted,train_series))

# Create a Plotly figure
fig = go.Figure()

# Add a trace for the predicted values
fig.add_trace(go.Scatter(
    x=predicted.pd_series().index,
    y=predicted.pd_series().values,
    name='predicted'
))

# Add a trace for the actual values
fig.add_trace(go.Scatter(
    x=testing_sc[0:Horizan].pd_series().index,
    y=testing_sc[0:Horizan].pd_series().values,
    name='actual'
))

# Update the traces to include hover info and mode
fig.update_traces(hoverinfo='text+name', mode='lines+markers')

# Update the figure layout to include a legend
fig.update_layout(legend=dict(y=0.1, traceorder='reversed', font_size=16))

# Update the figure layout to include a title
fig.update_layout(title="Horizon (Days): " + str(Horizan))

# Update the x-axis to include a range selector
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([dict(step="all")])
    )
)

# Show the figure
fig.show()


{'mae': 13.475525511635674, 'sampe': 38.496620156852835, 'mse': 326.4603052836397, 'rmse': 18.06821256471264, 'mape': 115.6506266325454, 'mase': 1.208790074157538}


# Forcasting AQI value of PM2.5 - Local Conditions

In [17]:
# Importing the full.csv file into pandas dataframe
df =  pd.read_csv('full.csv')

# Filter dataframe with specific site_name and aqs_param
df = multivariate_series(df,
                site_name = 'Los Angeles-North Main Street',
                aqs_param = 'PM2.5 - Local Conditions')

# Drop columns 'index' and duplicate dates
df = df.drop(columns=['index'])
df = df.drop_duplicates(subset='Date')

# Convert pandas dataframe to TimeSeries object
df_series_2 =  TimeSeries.from_dataframe(df,time_col='Date',freq='1D',fill_missing_dates=True,fillna_value=True)

# Split TimeSeries data into train and test sets
train_series ,testing_sc = df_series_2.split_after(0.8)

# Print the shape of train and test data
print("Shape of Training Data(In Days) : ",train_series.pd_dataframe().shape)
print("Shape of Testing Data(In Days) : ",testing_sc.pd_dataframe().shape)

# Initialize XGBModel object
xgb_pm = XGBModel(lags=12, 
            lags_past_covariates=None, 
            lags_future_covariates=None, 
            output_chunk_length=30, 
            add_encoders=None, 
            )

# Fit the model on train data
xgb_pm.fit(train_series)


Shape of Training Data(In Days) :  (1461, 1)
Shape of Testing Data(In Days) :  (365, 1)


In [18]:
# Define a list of horizons to be used for forecasting
Horizan = [15, 30, 90, 365]

# Create an empty list to store the evaluation results for each horizon
eval_list = []

# Loop through each horizon in the list
for horizan in Horizan:

    # Predict the AQI value for the given horizon using XGBoost
    predicted = xgb_pm.predict(horizan)

    # Calculate the evaluation metrics for the predicted values and actual values
    ret_eval_dic = Evaluation_matrics(testing_sc[0:horizan],predicted,train_series)
    # Store the evaluation results for the current horizon in a dictionary
    eval_dic = {"Horizon": horizan,
                "MAE": ret_eval_dic['mae'],
                "SMAPE": ret_eval_dic['sampe'],
                "MSE": ret_eval_dic['mse'],
                "RMSE": ret_eval_dic['rmse'],
                "MAPE": ret_eval_dic['mape']
                }

    # Add the evaluation results for the current horizon to the list of results
    eval_list.append(eval_dic)

# Create a Pandas dataframe from the evaluation results
ev_df = pd.DataFrame(eval_list)

# Display the evaluation results dataframe
ev_df


,Horizon,MAE,SMAPE,MSE,RMSE,MAPE
0,15,17.798566,35.896640,476.580669,21.830728,30.141003
1,30,17.243854,43.308430,429.026060,20.712944,44.391847
2,90,17.293717,43.019067,453.528572,21.296210,55.689407
3,365,17.346217,35.691710,580.765421,24.099075,59.555575


In [19]:
# Hyper parameter tunning
# from darts.model

## Evaluation of Horizan is 15 Day

In [20]:
# Import necessary libraries
# import xgboost as xgb
# import plotly.express as go

# Set the horizon value
Horizon = 15

# Predict using xgboost model
predicted = xgb_pm.predict(Horizon)

# Evaluate the performance of the model
# Evaluate the prediction using the Evaluation_matrics function, which likely calculates some performance metrics
print(Evaluation_matrics(testing_sc[0:Horizon],predicted,train_series))

# Create a plotly figure
fig = go.Figure()

# Add trace for predicted values
fig.add_trace(go.Scatter(
                            x=predicted.pd_series().index,
                            y=predicted.pd_series().values,
                            name='predicted'))

# Add trace for actual values
fig.add_trace(go.Scatter(
                            x=testing_sc[0:Horizon].pd_series().index,
                            y=testing_sc[0:Horizon].pd_series().values,
                            name='actual'))

# Update traces to display information on hover
fig.update_traces(hoverinfo='text+name', mode='lines+markers')

# Update the legend
fig.update_layout(legend=dict(y=0.1, traceorder='reversed', font_size=16))

# Update the title
fig.update_layout(title="Horizon(Days) : " + str(Horizon))

# Update x-axis to display rangeslider and rangeselector
fig.update_xaxes(rangeslider_visible=True,
                rangeselector=dict(
                buttons=list([dict(step="all")])
                ))

# Show the figure
fig.show()


{'mae': 17.79856554667155, 'sampe': 35.89664007073395, 'mse': 476.5806694960748, 'rmse': 21.830727644677232, 'mape': 30.141002571916815, 'mase': 1.4999079768046442}


## Evaluation of Horizan is 1 month

In [21]:
#Set the forecast horizon to 30
Horizan = 30

#Use XGBoost model to predict the next 30 values
predicted = xgb_pm.predict(Horizan)

#Evaluate the prediction using Evaluation_matrics function
print(Evaluation_matrics(testing_sc[0:Horizan],predicted,train_series))

# Create a plot using Plotly go.Figure()
fig = go.Figure()

#Add a trace for the predicted values
fig.add_trace(  go.Scatter(
                            x=predicted.pd_series().index,
                            y=predicted.pd_series().values,
                            name='predicted'))

#Add a trace for the actual values
fig.add_trace(  go.Scatter(
                            x=testing_sc[0:Horizan].pd_series().index,
                            y=testing_sc[0:Horizan].pd_series().values,
                            name='actual'))

#Update the traces to show text and name when hovering over the plot
fig.update_traces(hoverinfo='text+name', mode='lines+markers')

#Update the layout to show the legend with font size 16
fig.update_layout(legend=dict(y=0.1, traceorder='reversed', font_size=16))

#Update the layout to show the title with the forecast horizon
fig.update_layout(title="Horizan(Days) : "+ str(Horizan))

#Update the x-axis to show a range selector with all values selected by default
fig.update_xaxes(
                rangeslider_visible=True,
                rangeselector=dict(
                buttons=list([dict(step="all")])
                ))

#Show the plot
fig.show()


{'mae': 17.243854173024495, 'sampe': 43.30842974945796, 'mse': 429.0260596220425, 'rmse': 20.712944252858946, 'mape': 44.3918472703143, 'mase': 1.4531617369475187}


## Evaluation of Horizan is 3 Month

In [22]:
# Code to predict and visualize time-series data using XGBoost

# Set the horizon, which is the number of steps to predict in the future
Horizan = 30 * 3

# Use the XGBoost model to predict the time-series data for the given horizon
predicted = xgb_pm.predict(Horizan)

# Evaluate the model using evaluation metrics on the predicted and actual data
print(Evaluation_matrics(testing_sc[0:Horizan],predicted,train_series))

# Create the figure for visualization
fig = go.Figure()

# Add the predicted data as a trace
fig.add_trace(  go.Scatter(
                            x=predicted.pd_series().index,
                            y=predicted.pd_series().values,
                            name='predicted'))

# Add the actual data as a trace
fig.add_trace(  go.Scatter(
                            x=testing_sc[0:Horizan].pd_series().index,
                            y=testing_sc[0:Horizan].pd_series().values,
                            name='actual'))

# Update the traces to show hover info and display mode
fig.update_traces(hoverinfo='text+name', mode='lines+markers')

# Update the layout to include a legend
fig.update_layout(legend=dict(y=0.1, traceorder='reversed', font_size=16))

# Update the layout to include a title
fig.update_layout(title="Horizon (Days) : "+ str(Horizan))

# Update the x-axis to include a range selector
fig.update_xaxes(
                rangeslider_visible=True,
                rangeselector=dict(
                buttons=list([dict(step="all")])
                ))

# Show the figure
fig.show()


{'mae': 17.293716843922933, 'sampe': 43.0190668044648, 'mse': 453.52857234435265, 'rmse': 21.296210281276636, 'mape': 55.68940722640645, 'mase': 1.4573637282613263}


## Evaluation of Horizan is 1 Year

In [23]:

# Set the horizon to 30 years in months
Horizan = 30 * 12

# Fit the data to auto arima model
# auto_arima.fit(train_sc)

# Predict the values using xgboost
predicted = xgb_pm.predict(Horizan)

# Print the evaluation metrics by comparing the predicted and actual values
print(Evaluation_matrics(testing_sc[0:Horizan],predicted,train_series))

# Create a plotly figure object
fig = go.Figure()

# Add a trace for the predicted values
fig.add_trace(  go.Scatter(
                            x=predicted.pd_series().index,
                            y=predicted.pd_series().values,
                            name='predicted'))

# Add a trace for the actual values
fig.add_trace(  go.Scatter(
                            x=testing_sc[0:Horizan].pd_series().index,
                            y=testing_sc[0:Horizan].pd_series().values,
                            name='actual'))

# Update the trace information
fig.update_traces(hoverinfo='text+name', mode='lines+markers')

# Update the legend of the plot
fig.update_layout(legend=dict(y=0.1, traceorder='reversed', font_size=16))

# Update the title of the plot
fig.update_layout(title="Horizan(Days) : " + str(Horizan))

# Update the x-axis range selector
fig.update_xaxes(
                rangeslider_visible=True,
                rangeselector=dict(
                buttons=list([dict(step="all")])
                ))

# Show the plot
fig.show()


{'mae': 17.28842139508989, 'sampe': 35.3986453885192, 'mse': 580.519639031811, 'rmse': 24.09397516043816, 'mape': 58.96018020963378, 'mase': 1.4569174739873731}
